<a href="https://colab.research.google.com/github/Riddick4-droid/CHAT-BOT-Langchain-APPS/blob/main/RAG_APP_Advanced_Routing_Query_Construction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ROUTING AND QUERY RECONSTRUCTION

In [ ]:
##usually we find that queries are all mapped to a single knowledge source
##what if we have multiple knowledge sources from our rag system and we want
##the retriever to be very proactive in routing every users query to the right knowledge source
##in this notebook i intend on exploring query routing and reconstruction-metadata wise

QUERY ROUTING

In [ ]:
import os
import sys
import pandas as pd
import bs4
from langchain_community.document_loaders import WebBaseLoader,PyMuPDFLoader,Docx2txtLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableLambda
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from langchain.utils.math import cosine_similarity, cosine_similarity_top_k
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain.load import dumps,loads
from typing import Literal
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.retrievers import EnsembleRetriever
from typing import Any
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import userdata
langchain_api_key = userdata.get('LANGCHAIN_API_KEY')
openai_api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
##setting up environment variables
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.lanchain.com'
os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
os.environ['OPENAI_API_KEY'] = openai_api_key

In [ ]:
##load docx
loaded_docs = PyPDFLoader('/content/2023-Annual-Report-CalBank.pdf',
                          extract_images=False,
                          )
doc1 = loaded_docs.load()

In [ ]:
loaded_docs = PyPDFLoader('/content/pdoc.pdf',
                          extract_images=False,
                          )
doc2 = loaded_docs.load()

In [ ]:
##creating a class to define a basemodel schema to perform the logical routing
class RouteQuestions(BaseModel):
    datasource: Literal['financial_data', 'political_data'] #feel free to change the doc names to suit your use case

In [ ]:
##the above class uses a BaseModel to define a router
##the Literal function allows us to create a list of items and the Field function
##allows us to ensure that the items in the Literal function are adhered to

In [ ]:
##load the llm
llm_model = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.5)

#now we structure the output of the llm to adhere to the schema
structured_output = llm_model.with_structured_output(RouteQuestions)

In [ ]:
##defining prompt template with instructions for llm
system = """
You are an expert in logically understanding a user query and by this able to route the query to the right
information source. In this case, i want you to choose the right datasource for this query.
"""

##using the from_messages template to combine the system instruction and the question

##in a message like format
prompt = ChatPromptTemplate.from_messages([
    ('system',system),
    ('human','{question}')]
)

#display what the prompt object looks like specificall the input variables
print(prompt)

input_variables=['question'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\nYou are an expert in logically understanding a user query and by this able to route the query to the right\ninformation source. In this case, i want you to choose the right datasource for this query.\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]


In [ ]:
##define the router chain
router = prompt|structured_output

In [ ]:
##invoking
router.invoke({'question':'what is political theory'})

RouteQuestions(datasource='political_data')

In [ ]:
router.invoke({"question":"explain the balance sheet for the fiscal year"})

RouteQuestions(datasource='financial_data')

## make the routing more realistic

In [ ]:
'''create a vectorstore for the docs such that when a query is passed it will route to the
right vectorstore and retrieve the context'''


splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [ ]:
split1 = splitter.split_documents(doc1)
split2 = splitter.split_documents(doc2)

In [ ]:
##vectorstore for storing chunked embeddings

#document1
vectorstore1 = Chroma.from_documents(split1, embedding=OpenAIEmbeddings())

#document2
vectorstore2 = Chroma.from_documents(split2, embedding=OpenAIEmbeddings())

In [ ]:
#set up retriever
retriever1 = vectorstore1.as_retriever(similarity_score_threshold=0.8,k=3)
retriever2 = vectorstore2.as_retriever(similarity_score_threshold=0.8,k=3)

In [ ]:
#format the context documents
#for the context processing
def format_docs(docs):
    '''
    args:docs=takes in the retrieved context document page

    return: a formatted string
    '''
    return " ".join(d.page_content for d in docs)

In [ ]:
#creating chain for retrieved documents strings with langchain runnablelambda
context1 = retriever1 | RunnableLambda(format_docs)
context2 = retriever2 | RunnableLambda(format_docs)

In [ ]:
#test the retriever and context chain
context1.invoke('what is finance?')

'financial; it is a testament to your confidence in our ability \nto deliver our strategic results. We take this trust seriously \nand remain committed to maximizing shareholder value.\nI welcome you to the 2024 financial year with the firm \nassurance of excellent performance by our Bank.\nJoe Mensah\nChairman 123\nAnnual Report 2023PLC\nNotes T o The  Financial Statements (Continued)\n• Brokerage: subscribe for, underwrite, buy, hold, manage, and sell securities either on or off a stock exchange either as \nprincipals or agents. It also provides issuing house underwriting services and sponsorship to corporate clients.\n• Asset Management: provide asset management, investment portfolio management, cash management, money \nmanagement and other investment advisory services to institutional investors, businesses and high net worth individuals \nand manage mutual funds. \nManagement monitors the operating results of its business units separately for the purpose of making decisions about \

In [ ]:
import textwrap
def wrapped(text):
    '''purpose: finer display of text
    args: text : takes in a string

    returns: a wrapped string'''
    return textwrap.fill(text, fix_sentence_endings=True,replace_whitespace=True)

In [ ]:
response = context2.invoke('what is political theory and how does it affect the population?')
print(wrapped(response))

citizens to express our preferences and hold our elected
representatives to account.  In this  chapter we consider theories of
elite and mass opinion formation to understand how the  political
representation of our interests, beliefs and values works, or doesn’t
work, in practice.  We explore the relationships between values and
party choice and how these have changed  over time.  Finally, we
examine the Scottish and EU referendums, considering how they have
altered the relationships we have with the political parties and the
way we understand pub - lic opinion in the UK. Chapter 9 looks at how
the news media have a critical public service role in a democratic
society.  We show that ideally journalism will offer a platform for a
wide range of ideas to be  debated, which will help inform the public
on political affairs.  We examine whether the  news media enhances or
restricts pluralism in the UK political system, and we see that news
political analysts is to explore just how our politi

In [ ]:
print(wrapped(context2.invoke('what is balance sheet analysis for quarter 2')))

less dividend  -  -  (69,031)  (68,956) leaving a balance on retained
earnings carried forward of  (1,202,955) (1,159,113)  (522,677)
(488,045) The Directors consider the state of the Group and Bank’s
affairs to be satisfactory.  In December 2023, as part of their annual
Asset Quality Review (AQR), the Board of Directors conducted a
comprehensive  review of the Bank’s loan and advances portfolio.  This
review was prompted by the challenging macroeconomic conditions  in
Ghana over the past few years, characterized by high inflation and
elevated interest rates.  Many of our customers have  encountered
difficulties in servicing their debts in accordance with the agreed
terms.  Consequently, significant additional provisions  have been
made on the top 50 loans to accurately reflect the current economic
situation, resulting in the Bank recording losses  for the 2023
financial year-end.  20 Annual Report 2023PLC Managing Director’s
Report (Continued) In December 2023, as part of their annual

In [ ]:
##now we can define a function to help choose the right document
prompt = """
Using the given context:
---------
{context_str}
---------
Explain and correct any questions from the user. Ensure to stick to the context
Do not assume.
Question: {question}
Answer:
"""
template = ChatPromptTemplate.from_template(prompt)

print(template)

input_variables=['context_str', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context_str', 'question'], input_types={}, partial_variables={}, template='\nUsing the given context:\n---------\n{context_str}\n---------\nExplain and correct any questions from the user. Ensure to stick to the context\nDo not assume.\nQuestion: {question}\nAnswer:\n'), additional_kwargs={})]


In [ ]:
##check the output of the router
route = router.invoke('what is government?')
print(route)

datasource='political_data'


In [ ]:
##intelligent routing of the query to document
def logical_routing(inputs: Any) -> str:
    try:
        route = inputs['route']  # RouteQuestions object
        question = inputs['input']['question']  # Original question string
        datasource = route.datasource.lower() #get the routed data source
    except (KeyError, AttributeError):
        return "Error: Invalid result object or missing route/question"

    if 'financial_data' in datasource:
        print('Routing to financial data')
        rag_chain = (
            {
                'context_str': context1, #for the financial contexts
                'question': RunnablePassthrough()
            }
            | template
            | llm_model
            | StrOutputParser()
        )
        return rag_chain.invoke(question)
    elif 'political_data' in datasource:
        print('Routing to Politics document')
        rag_chain = (
            {
                'context_str': context2 , #contains political contexts
                'question': RunnablePassthrough()
            }
            | template
            | llm_model
            | StrOutputParser()
        )
        return rag_chain.invoke(question)
    else:
        return "I don't have the required knowledge"

In [ ]:
#configure the full rag chain passing both the router output and the question
full_chain = {'route': router, 'input': RunnablePassthrough()} | RunnableLambda(logical_routing)

In [ ]:
#test the chain by invoking a question
question = 'what number of years are registered in for the balance sheets and statement of financial position'

response = full_chain.invoke({'question': question})
print(wrapped(response))

Routing to financial data
The financial statements provided are for the year ended 31 December
2023. The balance sheets and statement of financial position are
registered for the year 2023.


In [ ]:
question = 'in what diverse ways does politics affect the society both nationally and internationally?'
response = full_chain.invoke({'question': question})
print(wrapped(response))

Routing to Politics document
In the context provided, politics affects society in diverse ways both
nationally and internationally.  Nationally, politics shapes how we
think about our pasts, reflects the diverse range of experiences of
citizens within the UK, and influences the flow of ideas and knowledge
within the political system.  It provides space for different
groupings within society to operate, be heard, and listened to,
allowing for a plurality of voices to contribute to good politics and
policy-making.  Embracing the rich diversity of experiences,
positions, attitudes, and beliefs within the nation-state is seen as
beneficial for solving problems effectively and enabling the UK to
evolve and adapt to global challenges.   Internationally, politics
plays a crucial role in shaping the UK's involvement in global issues
and how it interacts with other countries.  By continually questioning
and updating beliefs and knowledge, the UK can better navigate global
problems and contribut

## Semantic Routing

#### in this routing technique, we assume we do not have a data source and hence will use an llm as the routing destination
#### the task is to configure the llm so that it is able to assess the users the query and choose another llm prompt that has been set as the expert in the field of choice.

In [ ]:
##configuring prompts
##in this case i will use a financial expert and data analyst
#1.financial expert
financial_expert_prompt = """
You are a very experienced financial analyst.
You are great at answering questions and analyzing financial data in a concise and easy  to understand manner
When you don't know the answer just say **I DO NOT KNOW**
"""
#2. data analyst
data_expert_prompt = """
You are a very experienced political analyst.
You are great at answering questions and analyzing national and global political data in a concise and easy  to understand manner
When you don't know the answer just say **I DO NOT KNOW**
"""

In [ ]:
'''for this one there is no context and we want to leverage the llms pretrained knowledge
rather; we would leverage similarity measures like the cosine similarity or semanticsimilaritysearch
import similarity algorithm'''

#embedding function
embedder = OpenAIEmbeddings(model='text-embedding-3-small')

#coallate the templates
prompts = [financial_expert_prompt,data_expert_prompt]


In [ ]:
prompt_embeddings = embedder.embed_documents(prompts)

In [ ]:
len(prompt_embeddings)

2

In [ ]:
from tqdm.auto import tqdm
for i in tqdm(prompt_embeddings[:100],desc='Loading embeddings'):
    print(i)

Loading embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

[0.01675921306014061, -0.01998363807797432, -0.014106862246990204, 0.0475342757999897, 0.010986450128257275, 0.03154216334223747, -0.03624878451228142, 0.023663124069571495, 0.01951557584106922, 0.03996727615594864, 0.048730432987213135, -0.030996091663837433, 0.012553157284855843, -0.03463657200336456, 0.01397684495896101, 0.0394212044775486, -0.056895509362220764, -0.0213618203997612, -0.006107556167989969, 0.028265731409192085, 0.04438785836100578, 0.025171322748064995, -0.024222197011113167, 0.0026588509790599346, 0.0210757814347744, 0.0008434863411821425, -0.019138526171445847, -0.009861801750957966, -0.01366480439901352, 0.04204755276441574, 0.04108542576432228, -0.014691939577460289, -0.04896446317434311, 0.011825061403214931, -0.0013952467124909163, -0.004797633271664381, -0.03284233435988426, 0.030580036342144012, 0.032244257628917694, -0.055127277970314026, -0.01136349979788065, -0.007645009085536003, 0.046468134969472885, 0.0044303350150585175, 0.032114241272211075, -0.02761

In [ ]:
'''the goal is to use the cosine similarity function to calculate
the distance or in other words the
similarity between the predefined and embedded prompt template and the user query'''

#define a fuinction to automate the process
def similarity_checker(input,prompt_emb):
    '''
    purpose: measure cosine similarity between prompt embeddings and the user query
    args: input: this is the user's question
          prompt_emb: the predefined prompt embeddings
    returns: the index in the prompt with the highest similarity score'''
    query_embeddings = embedder.embed_query(input['question'])

    #use the similarity checker
    similarity = cosine_similarity([query_embeddings],prompt_emb)

    #apply an argmax to return which index is highest
    index_doc = similarity.argmax()

    return index_doc

In [ ]:
similarity_index = similarity_checker({'question':'what is the state and adverse effect of politics'},
                                     prompt_embeddings)

In [ ]:
print(f'debug: routed to index {similarity_index}')

debug: routed to index 1


In [ ]:
##lets see what the indices returned mean
print(f'routing to index {similarity_index} for: ')
prompts[similarity_index]

routing to index 1 for: 


"\nYou are a very experienced political analyst.\nYou are great at answering questions and analyzing national and global political data in a concise and easy  to understand manner\nWhen you don't know the answer just say **I DO NOT KNOW**\n"

In [ ]:
##now lets route it
def router(input: int):
    if input==0:
        print(f'choosing prompt template {prompts[input]}')
        chosen_prompt = prompts[input]
        return chosen_prompt
    elif input ==1:
        print(f'choosing prompt template {prompts[input]}')
        chosen_prompt = prompts[input]
        return chosen_prompt
    else:
        return 'Not recognized'
    #return PromptTemplate.from_template(chosen_prompt)

In [ ]:
temp_context = router(similarity_checker({'question':'what is the difference between a balance sheet and statement of financial position?'},
                                       prompt_embeddings))

choosing prompt template 
You are a very experienced financial analyst.
You are great at answering questions and analyzing financial data in a concise and easy  to understand manner
When you don't know the answer just say **I DO NOT KNOW**



In [ ]:
##use the formatter
def format(text):
    return ''.join(t for t in text)

def wrapper(text):
    return textwrap.fill(text.strip('\n'),width=500,replace_whitespace=True,expand_tabs=False,
                         fix_sentence_endings=True)

In [ ]:
#configure llm chat model
llm = ChatOpenAI(model='gpt-5-mini',verbose=True,temperature=1)

In [ ]:
#use the returned router prompt as context
final_prompt = """
Given this context:
-----------------
{instruction}.
-----------------
Leverage your full expertise on the matter at hand

Your answer should be indepth and precise touching on all key areas of the question:
-----------------
{question}
-----------------
Answer:

"""

In [ ]:
prompt = PromptTemplate.from_template(final_prompt)

In [ ]:
prompt

PromptTemplate(input_variables=['instruction', 'question'], input_types={}, partial_variables={}, template='\nGiven this context:\n----------------- \n{instruction}. \n-----------------\nLeverage your full expertise on the matter at hand\n\nYour answer should be indepth and precise touching on all key areas of the question: \n-----------------\n{question}\n-----------------\nAnswer:\n\n')

In [ ]:
rag_chain = ({'question':RunnablePassthrough()}
             |RunnableLambda(lambda x : {'instruction':router(similarity_checker(x['question'],prompt_embeddings)),
                                         'question': x['question']})
             |prompt
             |llm
             |StrOutputParser())

In [ ]:
wrapper(rag_chain.invoke({'question':'what do you think makes up a poltical party and how does it operate?'}))

choosing prompt template 
You are a very experienced political analyst.
You are great at answering questions and analyzing national and global political data in a concise and easy  to understand manner
When you don't know the answer just say **I DO NOT KNOW**



'Short answer A political party is an organized group that seeks to gain and exercise political power by nominating candidates, winning elections, forming governments or opposition, and shaping public policy.  Its composition and operation combine formal institutions (offices, rules, leadership, funding) with informal networks (activists, patronage ties, media presence) to perform the tasks of representation, recruitment, mobilization and governance.  What makes up a political party — core\ncomponents - Membership and supporters: formal members who pay dues/participate and a broader base of voters and sympathizers.  Membership provides activists, volunteers and legitimacy.  - Leadership and office-holders: elected leaders, party executives, campaign directors and those who hold government or legislative positions.  - Organizational structure: national, regional and local branches; party committees; youth and women’s wings; electoral units.  Includes staff, volunteers and offices.\n- Ce

In [ ]:
wrapper(rag_chain.invoke({'question':'what do you think makes up statement of financial position?'}))

choosing prompt template 
You are a very experienced financial analyst.
You are great at answering questions and analyzing financial data in a concise and easy  to understand manner
When you don't know the answer just say **I DO NOT KNOW**



'Short answer The statement of financial position (balance sheet) shows a company’s resources (assets), claims on those resources by outsiders (liabilities), and owners’ claims (equity) at a single point in time.  Its primary sections are: Assets, Liabilities, and Equity.  Detailed breakdown (what makes it up)  1. Assets - Current assets (expected to be realized/consumed within 12 months or the operating cycle):   - Cash and cash equivalents   - Short-term investments/marketable securities   -\nTrade and other receivables (net of allowances)   - Inventories   - Prepayments (prepaid expenses)   - Current tax assets - Non‑current (non‑current/long‑term) assets:   - Property, plant & equipment (PPE) — net of accumulated depreciation   - Investment property   - Intangible assets (patents, trademarks, customer lists) and goodwill   - Long‑term investments (equity or debt securities held long term)   - Deferred tax assets   - Right‑of‑use assets (leases under IFRS 16/ASC 842)   - Biological\